<a name="Top"></a>
<p><span style="font-size: 30px;"><strong>BQL</strong></span><strong><br></strong><span style="text-align: inherit; font-size: 28px;"><strong><span style="color: orange;">Fixed Income - Universe</span></strong></span></p>

1. [Fixed Income Index Constitutents](#fi_memb)
2. [Fixed Income Universe Functions](#fi_univ)
3. [Fixed Income Database and Screening](#fi_screening)
4. [Grouping and Aggregation](#fi_agg)

In [3]:
import bql
from functools import reduce # To be used later in Screening

bq = bql.Service()


<a name="fi_univ"></a>
<p><strong><span style="color: orange; font-size: 24px;">Fixed Income Universe Functions</span></strong></p>

<p><span style="color: rgb(247, 218, 100); font-size: 20px;"><strong>Issuer Debt Chain</strong></span></p>

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 20%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
            <td style="width: 65%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Overrides/Parameters</span></strong></div>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Issuer's bonds</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BONDS()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">CONSOLIDATEDUPLICATES, USEACTIVEONLY, ISSUEDBY, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES, INCLUDENONBLOOMBERGVERIFIEDBONDS, DATES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Issuer's loans</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">LOANS()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">USEACTIVEONLY, ISSUEDBY, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES, INCLUDESTRIPS, DATES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Issuer's municipal bonds</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MUNICIPALS()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">USEACTIVEONLY, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Issuer's mortgage bonds</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MORTGAGES()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">CONSOLIDATEDUPLICATES, USEACTIVEONLY, INCLUDEPRIVATESECURITIES</td>
        </tr>        
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Issuer's debt</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">DEBT()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">CONSOLIDATEDUPLICATES, USEACTIVEONLY, ISSUEDBY, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES, INCLUDENONBLOOMBERGVERIFIEDBONDS, INCLUDESTRIPS</td>
        </tr>  
    </tbody>
</table>

## `bq.univ.bonds()` / `bq.univ.loans()` 
<p><span style="font-size: 20px;"><strong>Feature</strong></span></p>

- You can get the bond chain from the issuer's ticker.
- By using 'ISSUEDBY', you can specify the output based on the capital structure

It is possible to obtain debt data based on the CAST capital structure from issuers in the following: 

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 30%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Parameter</span></strong></div>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Details</span></strong></div>
        </tr>
        <tr>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">ENTITY</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Debt of specified entity only</td>
        </tr>
        <tr>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">ENTITY_DIRECT_SUBS</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Debt of specified entity and direct subsidiaries</td>
        </tr>
        <tr>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">ENTITY_SUBS</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Debt of specified entity and all subsidiaries</td>
        </tr>
        <tr>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">CREDIT_FAMILY</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Debt of guarantor closest to the CAST parent for the specified entity</td>
        </tr>
        <tr>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">CAST_PARENT_SUBS</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Debt of parent company and all subsidiaries</td>
        </tr>
    </tbody>
</table>




<img src="img/CAST.PNG">

In [4]:
# bonds of DBS bond issued by the entity and all its subsidiaries
issuer = ['DBS SP Equity']
bonds = bq.univ.bonds(issuer,issuedby='entity_direct_subs')

name = bq.data.id().group(bq.data.LONG_COMP_NAME()).count()['value']

req = bql.Request(bonds,{'count':name})
res = bq.execute(req)
res[0].df().head()

,count
ID,
DBS Bank Ltd,109
DBS Group Holdings Ltd,12


<p><span style="color: rgb(247, 218, 100); font-size: 20px;"><strong>Corporate Structure</strong></span></p>

You can extract bonds issued by coompanies based on its capital structure (CAST) or related securities (RELS).

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 40%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
            <td style="width: 45%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Overrides/Parameters</span></strong></div>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Bond Issuer</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">ISSUEROF()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">DATES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Parent of  Bond Issuer</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PARENT()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TYPE</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Equity Pricing ticker within Corporate Structure</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">EQUITYPRICINGTICKER()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"></td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Fundamental ticker (where closest fundamentals are reported)</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">FUNDAMENTALTICKER()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"></td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">ESG reporting entity within Corporate Structure</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">ESGTICKER()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"></td>
        </tr>
    </tbody>
</table>

## `bq.univ.issuerof()` / `bq.univ.parent()`
<p><span style="font-size: 20px;"><strong>Feature</strong></span></p>

You can get the capital structure in the following codes:
- `bq.univ.issuerof()`：to get the issuer  of a bond
- `bq.univ.parent()`：to get the parent of the bond's issuer
- `bq.univ.parent().parent()`：to move up the chain to the parent of the parent (if available) 
- `bq.univ.parent(type='ultimate')`：to get the ultimate parent of the bond's issuer




In addition to the above, database universes are also available. For more details, please check [Fixed Income Database and Screening](#fi_screening).

In [5]:
from collections import OrderedDict

bond = ['AZ066734 Corp','BP998214 Corp']

flds= OrderedDict()

name = bq.data.long_comp_name()

flds['Issuer'] = name.value(bq.univ.issuerof(bond),mapby='lineage')['value']
flds['Parent Company'] = name.value(bq.univ.parent(bond),mapby='lineage')['value']
flds['Parent of Parent Company'] = name.value(bq.univ.parent(bond).parent(),mapby='lineage')['value']
flds['Ultimate Parent Company'] = name.value(bq.univ.parent(bond,type='ultimate'),mapby='lineage')['value']


req = bql.Request(bond,flds)
res = bq.execute(req)
bql.combined_df(res)

,Issuer,Parent Company,Parent of Parent Company,Ultimate Parent Company
ID,,,,
AZ066734 Corp,BNZ International Funding Ltd/London,BNZ International Funding Ltd,Bank of New Zealand,National Australia Bank Ltd
BP998214 Corp,ASB Bank Ltd,ASB Holdings Ltd,Commonwealth Bank of Australia,Commonwealth Bank of Australia


<p><span style="color: rgb(247, 218, 100); font-size: 20px;"><strong>SRCH Search</strong></span></p>

## `bq.univ.screenresults(screen_name)`

SCREENRESULTS() can be used to retrieve a list of bonds saved in Bond Search (SRCH) and other fixed income screening tool (MTGS, PSCH, MSRC, LEAG etc). 


<p><span style="font-size: 20px;"><strong>Feature</strong></span></p>

- To apply a specific screen/search saved within SRCH, you will need to add the @ symbol to the name of the screen within the `screen_name` paranmeter (eg. "screen_name='@COCO'") 
- Saved list names are case sensitive (must be an exact match)



In [6]:
#pulling from SRCH screen name '@GREEN'
screen = '@GREEN'
univ = bq.univ.screenresults(type='SRCH',screen_name=screen)

sector = bq.data.classification_name(classification_scheme='bclass',classification_level='1')
count = bq.data.id().group(sector).count()['value']

req = bql.Request(univ,count,with_params=dict(mode='cached'))
res = bq.execute(req)
res[0].df()

,"COUNT(GROUP(ID(),CLASSIFICATION_NAME(classification_scheme='bclass',classification_level='1'))).VALUE"
ID,
Corporate,4876
Government-Related,2976
Securitized,172
Treasury,125


<a name="fi_screening"></a>
<p><strong><span style="color: orange; font-size: 24px;">3. Fixed Income Database and Screening</span></strong></p>

## `bq.univ.bondsuniv()` 
<p><span style="font-size: 20px;"><strong>Feature</strong></span></p>

- Similar to SRCH, you can screen for bonds using conditions and filters on our bonds database.
- By using `bondsuniv(types=['active'])`, this allows you to retrieve active bonds only. 
- By using `bql.Request(with_params=dict(dates='YYY-MM-DD'))`,  you can reference the search as of a specific date.
- By using `bq.func.filter()`, you can apply screening conditions and filters on the search.

**Available from January 1, 2015 onwards

Fixed Income Database Functions

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 20%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
            <td style="width: 65%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Overrides/Parameters</span></strong></div>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Corporate/Government Bonds Universe</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BONDSUNIV()</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TYPES, CONSOLIDATEDUPLICATES, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES, INCLUDENONBLOOMBERGVERIFIEDBONDS, DATES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Loans Universe</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">LOANSUNIV</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TYPES,INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES, INCLUDENONBLOOMBERGVERIFIEDBONDS, INCLUDESTRIPS, DATES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Municipal Bonds Universe</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MUNICIPALSUNIV</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TYPES, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Mortgages Universe</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MORTGAGESUNIV</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TYPES, CONSOLIDATEDUPLICATES, INCLUDEPRIVATESECURITIES</td>
        </tr>        
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">All Debt Universe</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">DEBTUNIV</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TYPES, CONSOLIDATEDUPLICATES, INCLUDEPRELIMINARYSECURITIES, INCLUDEPRIVATESECURITIES, INCLUDENONBLOOMBERGVERIFIEDBONDS, INCLUDESTRIPS</td>
        </tr>  
    </tbody>
</table>


With only one condition

Example：
```python
condition1 = bq.data.cntry_of_risk() == 'JP'
filtered_univ = univ.filter(condition1)
```

With two conditions, use `and_()` function

Example：
```python
condition1 = bq.data.cntry_of_risk() == 'JP'
condition2 = bq.data.classification_name(classification_scheme='bics',classification_level='1') == 'Information Technology'

combined_condition = bq.func.and_(condition1, condition2) 
filtered_univ = univ.filter(combined_condition)

```

For either of two conditions, use `or_()` function

Example：
```python
condition1 = bq.data.issue_dt() > '2022-01-01'
condition2 = bq.data.yield_(yield_type='YTM') <= 3

final_condition = bq.func.or_(condition, condition2)
filtered_univ = univ.filter(final_condition)

```



For 3 or more conditions, use `reduce()` function 

`reduce()` to iterate functions

In [7]:
condition1 = bq.data.cntry_of_risk() == 'JP'
condition2 = bq.data.yield_(yield_type='YTM') <= 3
condition3 = bq.data.classification_name(classification_scheme='bics',classification_level='1') == 'Information Technology'

In [8]:
reduce(bq.func.and_,[condition1]) #condition2,condition3

<BqlItem(name=EQUALS,CNTRY_OF_RISK(),<bqlmetadata.literals.AnyScalarLiteral.Value object at 0x7f77fb5adc70>)>

In [9]:
# Screen for bonds that meets the following conditions and obtain YTM for the last 30 days 
# 1. Issuer is JGB
# 2. Fixed Coupon
# 3. Maturity date is within 1 year
univ= bq.univ.bondsuniv('active')

security = 'JGB'
ticker = bq.data.ticker()
cpn_typ = bq.data.cpn_typ()
mat_year = bq.data.maturity().year()['value']

condition = reduce(bq.func.and_,[
    ticker==security,
    cpn_typ=='FIXED',
    mat_year <= bq.func.today().year()+1
    ])

data = bq.data.yield_(yield_type='YTW')

filtered_univ = univ.filter(condition)

req = bql.Request(filtered_univ,data,with_params=dict(mode='cached'))

res = bq.execute(req)
res[0].df().head()

,DATE,YIELD(yield_type='YTW')
ID,,
ZI638626 Corp,2024-09-19,2.220000e-01
BH726449 Corp,2024-09-19,1.520000e-01
BN506501 Corp,2024-09-19,2.940000e-01
ZM122842 Corp,2024-09-19,9.900000e-02
EK469005 Corp,2024-09-19,1.000000e-07


<a name="fi_agg"></a>
<p><strong><span style="color: orange; font-size: 24px;">4. Grouping and Aggregation</span></strong></p>

## `bq.func.group()` - Grouping

You can group the universe by a specific grouping type (eg. maturity, country of risk) and aggregate the results by applying a calculation method (eg. average, min, max etc) 
After grouping with ```.group()```, you can add a calculation method like ```.group(groupby).avg()``` to get the group average. 

<p><span style="font-size: 20px;"><strong>Feature</strong></span></p>

- You can group by country, industry etc using the `groupby` parameter.
- Example: `d.pe_ratio().group(groupby=d.cntry_of_domicile()).avg()`
- The output after grouping will be the value of the group applied, not the individual bond. 


Main functions used for aggregation

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 50%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Count</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">COUNT()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Average</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">AVG()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Weighted Average</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">WAVG()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Median</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MEDIAN()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Standard Deviation</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">STD()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Max</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MAX()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Min</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MIN()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Sum</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">SUM()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Bins</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BINS()</td> 
        </tr>
    </tbody>
</table>



In [10]:
# Pulling bonds within Energy sector in Russia
# How to get the average spread and the number of issues grouped by maturity buckets
univ = bq.univ.bondsuniv('active')

country = bq.data.cntry_of_risk() 
sector = bq.data.classification_name(classification_scheme='bclass',classification_level='3')

#definint the maturity buckets
mat_year = bq.data.maturity_years()
bins = [3,5,10,20]
bin_names = ['0-3','03-05','05-10','10-20','20+']
mat_year_bins = bq.func.bins(mat_year,bins=bins,bin_names=bin_names)
z_spread = bq.data.spread(spread_type='z')

#average spread per maturity bucket
avg_spread_by_bin = z_spread.group(mat_year_bins).avg()
count_by_bin = z_spread.group(mat_year_bins).count()

items = {'Count':count_by_bin,'Average Spread':avg_spread_by_bin}

condition1 = sector == 'Energy'
condition2 = country == 'RU'

final_condition = bq.func.and_(condition1,condition2)
filtered_univ = univ.filter(final_condition)

req = bql.Request(filtered_univ,items,with_params=dict(mode='cached',dates='2022-02-01'))

res = bq.execute(req)
bql.combined_df(res)

,DATE,ORIG_IDS,"BINS(MATURITY_YEARS(),BINS=[3,5,10,20],BIN_NAMES=['0-3','03-05','05-10','10-20','20+'])",Count,Average Spread
ID,,,,,
0-3,2022-02-01,None,0-3,12,341.632225
03-05,2022-02-01,None,03-05,3,222.347400
05-10,2022-02-01,None,05-10,14,117.716139
NullGroup,2022-02-01,None,None,10,79.301321


## `bq.func.ungroup()` - Ungrouping

<p><span style="font-size: 20px;"><strong>Feature</strong></span></p>

- Ability to ungroup results and view aggregated values at the individual bond level.
- Example：`d.pe_spread().group(groupby=d.cntry_of_risk()).avg().ungroup()` will get the country of risk average yield at the bond level

Example：

How to get sector average YTM displayed on individual bond level?



In [11]:
#retrieving memberse of the index
univ = bq.univ.members('I02923JP index')

sector = bq.data.classification_name(classification_scheme='bclass', classification_level='2')
ytm = bq.data.yield_(yield_type='YTM',pricing_source='BGN')['value']

#average yield by sector, ungroup to see individual securities
sector_avg_ytm = ytm.group(sector).avg().ungroup()
comparison = ytm > sector_avg_ytm

filtered_univ = univ.filter(ytm!=bql.NA)

items = {'YTM':ytm,'Average YTM':sector_avg_ytm,'Above Sector Average':comparison}

req = bql.Request(filtered_univ,items,with_params=dict(mode='cached'))
res = bq.execute(req)
bql.combined_df(res).head()

,YTM,Average YTM,Above Sector Average
ID,,,
BP069204 Corp,0.523,0.876867,False
AS999481 Corp,0.495,0.667582,False
AT476669 Corp,1.352,1.088784,True
ZP932943 Corp,0.581,0.667582,False
BQ447894 Corp,0.378,1.027592,False


## <p><span style="font-size: 20px;"><strong>Main functions used for ungroup()</strong></span></p>

- The list below is a simplified function of　`d.item().group().avg().ungroup()`
- Example：`groupavg()` gives the same calculation results as `group().avg().ungroup()`

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 50%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Count</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPCOUNT()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Average</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPAVG()</td>
        </tr>        
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Weighted Average</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPWAVG()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Median</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPMEDIAN()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Standard Deviation</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPSTD()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Max</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPMAX()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Min</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPMIN()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Sum</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPSUM()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Z-Score</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPZSCORE()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Cut</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPCUT()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Rank</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPRANK()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Group Sort</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">GROUPSORT()</td> 
        </tr>
    </tbody>
</table>

Example：

How to get OAS of bonds of TESLA US Equity in quintiles?

```python
univ = u.bonds('TSLA US Equity')

oas = d.spread(spread_type='OAS')['value']
oas_percentile = oas.groupcut(n=5)['value']

req = bql.Request(univ,oas_percentile,with_params=dict(mode='cached'))
res = bq.execute(req)
bql.combined_df(res)

```

In [12]:
univ = bq.univ.bonds('TSLA US Equity')

oas = bq.data.spread(spread_type='OAS')['value']
oas_percentile = oas.groupcut(n=5)['value']

req = bql.Request(univ,oas_percentile,with_params=dict(mode='cached'))
res = bq.execute(req)
bql.combined_df(res)


,"GROUPCUT(SPREAD(spread_type='OAS').VALUE,n=5).VALUE"
ID,
EK910129 Corp,5.0
EK845482 Corp,3.0
EK795706 Corp,3.0
UV322213 Corp,4.0
JV457767 Corp,3.0
JV296263 Corp,5.0
AF269003 Corp,4.0
EK795712 Corp,3.0
EK812329 Corp,3.0
